In [79]:
import configparser
import json
import os
import time
import requests
import pandas as pd

In [80]:
config = configparser.ConfigParser()
config.read("../api_token.ini")
API_KEY = config.get("ALPHAVANTAGE", 'alpha_vantage_api_key')

In [81]:
def get_historical_data(symbol, market, api_key):
    api_url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    return raw_df  


In [82]:
list_of_coins = pd.read_csv('../data/digital_currency_list.csv')
list_of_coins

,currency code,currency name
0,1ST,FirstBlood
1,2GIVE,GiveCoin
2,808,808Coin
3,AAVE,Aave
4,ABT,ArcBlock
...,...,...
568,ZEN,ZenCash
569,ZET,Zetacoin
570,ZIL,Zilliqa
571,ZLA,Zilla


In [92]:
zen = get_historical_data('ZEN', 'USD', API_KEY)

In [93]:
zen

{'Meta Data': {'1. Information': 'Daily Prices and Volumes for Digital Currency',
  '2. Digital Currency Code': 'ZEN',
  '3. Digital Currency Name': 'ZenCash',
  '4. Market Code': 'USD',
  '5. Market Name': 'United States Dollar',
  '6. Last Refreshed': '2021-11-06 00:00:00',
  '7. Time Zone': 'UTC'},
 'Time Series (Digital Currency Daily)': {'2021-11-06': {'1a. open (USD)': '85.41000000',
   '1b. open (USD)': '85.41000000',
   '2a. high (USD)': '86.27000000',
   '2b. high (USD)': '86.27000000',
   '3a. low (USD)': '85.04000000',
   '3b. low (USD)': '85.04000000',
   '4a. close (USD)': '85.75000000',
   '4b. close (USD)': '85.75000000',
   '5. volume': '2448.36000000',
   '6. market cap (USD)': '2448.36000000'},
  '2021-11-05': {'1a. open (USD)': '84.64000000',
   '1b. open (USD)': '84.64000000',
   '2a. high (USD)': '88.69000000',
   '2b. high (USD)': '88.69000000',
   '3a. low (USD)': '83.20000000',
   '3b. low (USD)': '83.20000000',
   '4a. close (USD)': '85.48000000',
   '4b. close

In [85]:
def get_current_rate(symbol, market, api_key):
    api_url = f'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency={symbol}&to_currency={market}&apikey={api_key}'
    raw_df = requests.get(api_url).json()
    return raw_df  


In [95]:
i = 0
while i <= 5:
    print(get_current_rate("BTC", "PLN", API_KEY))
    i += 1
    time.sleep(15)

{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'BTC', '2. From_Currency Name': 'Bitcoin', '3. To_Currency Code': 'PLN', '4. To_Currency Name': 'Polish Zloty', '5. Exchange Rate': '241308.05541900', '6. Last Refreshed': '2021-11-06 20:03:46', '7. Time Zone': 'UTC', '8. Bid Price': '241308.05541900', '9. Ask Price': '241308.09512600'}}
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'BTC', '2. From_Currency Name': 'Bitcoin', '3. To_Currency Code': 'PLN', '4. To_Currency Name': 'Polish Zloty', '5. Exchange Rate': '241322.94554400', '6. Last Refreshed': '2021-11-06 20:04:02', '7. Time Zone': 'UTC', '8. Bid Price': '241322.94554400', '9. Ask Price': '241322.98525100'}}
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'BTC', '2. From_Currency Name': 'Bitcoin', '3. To_Currency Code': 'PLN', '4. To_Currency Name': 'Polish Zloty', '5. Exchange Rate': '241322.94554400', '6. Last Refreshed': '2021-11-06 20:04:02', '7. Time Zone': 'UTC', '8. Bid Price': '2